# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie 1 - tokenizacja (12 pkt)

Jedną z nowoczesnych technik tokenizacji jest BPE - byte-pair encoding [1]. Technika ta polega na podzielenie słów na częste podsłowa (morfemy). W przeciwieństwie do podejść lingwistycznych, wymagających reguł tworzenia morfemów, BPE wyznacza je automatycznie poprzez wyznaczenie najczęstszych przylegających do siebie sekwencji znaków które występują obok siebie.

Algorytm przebiega w następujących krokach.
1. Podziel wszystkie słowa na symbole (początkowo pojedyncze znaki)
2. Wyznacz najczęściej występującą obok siebie parę symboli 
3. Stwórz nowy symbol będący konkatenacją dwóch najczęstszych symboli.

Uwaga 1: każde słowo zakończone jest specjalnym symbolem końca wyrazu.

Uwaga 2: tworzenie nowego symbolu nie powoduje usuniecie starego tj. zawsze jednym z możliwych symboli jest pojedynczy znak, ale jeśli można to stosujemy symbol dłuższy.

Przykład: korpus w którym występuje ,,ala'' 5 razy i ,,mama 10 razy''
1. Dzielimy słowa na symbole ,,a l a END'' ,,m a m a END''  gdzie END jest symbolem końca wyrazu.
2. Najczęstsza para obok siebie to ,,m a'' (20) razy
3. Nowy symbol ,,ma''
4. Nowy podział ,,a l a END'' ,,ma ma END''
5. Najczęstsza para ,,ma ma'' (10) razy
6. Nowy symbol ,,mama''
7. Nowy podział ,,a l a END'' ,,mama END''
8. itd.

W pliku ,,brown_clusters.tsv'' pierwsza kolumna to identyfikator skupienia (nie używamy w tym zadaniu), druga kolumna to wyrazy, a trzecia to ich liczności w pewnym korpusie tweetów. Zaimplementuj technike BPE na tych słowach.

Zaimplementuj algorytm BPE wykonujący `number_of_iterations` iteracji (łączeń symboli).

[1] Sennrich, R., Haddow, B., and Birch, A. (2016). Neural machine translation of rare words with subword units. In ACL 2016.

In [83]:
import pandas as pd
import numpy as np
from collections import Counter
brown_df = pd.read_csv('brown_clusters.tsv', sep='\t', header=0, names=['cluster', 'word', 'count'])

END_SYMBOL = 'END'
number_of_iterations = 10


def concatenate_words(splitted_word, first_elem, second_elem, concatenated):
    # Preapre output
    result = list()
    
    # Last seen element
    last_seen_elem = None
    
    for elem in splitted_word:
        if last_seen_elem is not None:
            # We must check both elements
            if last_seen_elem == first_elem and elem == second_elem:
                # Add concatenated word to result
                result.append(concatenated)
                
                # Clear last element to prevent duplications in replacement
                last_seen_elem = None
                
                # Check next element
                continue
            else:
                # Store previous element
                result.append(last_seen_elem)
            
        # Update last seen element
        last_seen_elem = elem
        
    # Add last element if not paired
    if last_seen_elem is not None:
        result.append(last_seen_elem)
        
    # Return joined list
    return result


def output(df):
    """
    Generate output as list of words, each with <END> tag
    """
    return df['word'].apply(lambda word: ' '.join(word) + ' ' + END_SYMBOL).tolist()
    

def preform_bpe(brown_df, number_of_iterations):
    """
    Funkcja przyjmuje ramkę w formacie analogicznym do obiektu brown_df (wczytany wyżej)
     oraz liczbę iteracji.
    Wyjściem funkcji powinna być lista słów z poszczególnymi tokenami/symbolami oddzielonymi spacją.
    Za znak końca wyrazu przyjmij END. 
    """
    # Copy dataframe
    df = brown_df.copy()
    
    # Split each character
    df['word'] = df['word'].astype(str).apply(lambda word: list(word))
    
    # Do it n-times
    for _iteration in range(number_of_iterations):
        # Vocabulary counter - we will use it to find the best pair
        vocabulary = Counter()

        # Iterate over elements in corpus
        for row in df.itertuples():
            last_element = None
            for element in row.word:
                if last_element is not None:
                    # We already seen element - can generate new pair
                    vocabulary[(last_element, element)] += row.count

                # Update last seen element
                last_element = element
            
        # Now we must ensure at least one pair found
        if not vocabulary:
            return output(df)

        # We can find maximum element
        ((first_elem, second_elem), cardinality) = vocabulary.most_common(1)[0]
        concatenated = first_elem + second_elem
        print(first_elem + ' AND ' + second_elem + ' ====> ' + concatenated)
    
        # Update words - join group 
        df['word'] = df['word'].apply(concatenate_words, args = (first_elem, second_elem, concatenated))
    
    # Return dataframe
    return output(df)

Test implementacji:

In [84]:
from nose.tools import assert_list_equal
data = {'cluster': range(2), 'word':['ala', 'mama'], 'count': [5,10]}
df = pd.DataFrame (data, columns = ['cluster', 'word', 'count'])
vocab = preform_bpe(df, 1)
assert_list_equal(vocab, ['a l a END', 'ma ma END'])

m AND a ====> ma


Spraw aby Twoja implementacja wypisywała kolejne łączone ze sobą symbole i uruchom Twoją funkcję na np. 50 iteracji, obserwując jakie tokeny są tworzone.

In [85]:
preform_bpe(brown_df, 50)

i AND n ====> in
t AND h ====> th
a AND n ====> an
e AND r ====> er
o AND n ====> on
o AND u ====> ou
r AND e ====> re
. AND . ====> ..
a AND t ====> at
t AND o ====> to
in AND g ====> ing
i AND t ====> it
th AND e ====> the
s AND t ====> st
< AND @ ====> <@
<@ AND M ====> <@M
<@M AND E ====> <@ME
<@ME AND N ====> <@MEN
<@MEN AND T ====> <@MENT
<@MENT AND I ====> <@MENTI
<@MENTI AND O ====> <@MENTIO
<@MENTIO AND N ====> <@MENTION
<@MENTION AND > ====> <@MENTION>
o AND r ====> or
m AND e ====> me
l AND l ====> ll
i AND s ====> is
e AND n ====> en
a AND r ====> ar
l AND e ====> le
y AND ou ====> you
o AND w ====> ow
h AND a ====> ha
c AND o ====> co
e AND d ====> ed
a AND y ====> ay
e AND s ====> es
< AND U ====> <U
<U AND R ====> <UR
<UR AND L ====> <URL
<URL AND - ====> <URL-
an AND d ====> and
c AND h ====> ch
a AND s ====> as
l AND o ====> lo
b AND e ====> be
v AND e ====> ve
a AND l ====> al
o AND f ====> of
w AND h ====> wh


['\\ i END',
 '/ i / END',
 'to d ay - i END',
 'n ow i END',
 '# you e v er END',
 'i f in a ll y END',
 '「 i END',
 '- i - END',
 'in e v a END',
 '» i END',
 'wh at t ay a END',
 'i i i i i i i i i i END',
 '\ue6d1 END',
 'i k in d a END',
 'lo l - i END',
 'i a c t u a ll y END',
 'w a d d y a END',
 '# as l on g as you END',
 'd o you END',
 '\u200e \u200b i END',
 'i ̇ END',
 'ï END',
 '# lo l at g i r l s wh o END',
 '# r t i f you END',
 'i j st END',
 '« i END',
 '• i END',
 'wh o d a END',
 'w ha d y a END',
 ') i END',
 '+ i END',
 '# you r f a c e m a k es me END',
 'i i i i i i i i END',
 '` i END',
 'i i i i i i i END',
 'i al re a d y END',
 '_ i END',
 '# you m a k e me END',
 '* i END',
 '| i END',
 '# u r b o y f r i en d e v er END',
 'wh en i END',
 'ι END',
 "d on ' t c ha END",
 "wh o ' d a END",
 'd you END',
 'w ha d d ay a END',
 'i on l y END',
 'i j u s s END',
 'i al w ay s END',
 'i i i i i END',
 'd on c ha END',
 '( i END',
 "d ' y a END",
 'ı END',
 '# u

- Jakie angielskie słowo jako pierwsze dostało swój własny token?

in

- Jakie są zalety korzystania z tokenizacji BPE w kontekście tworzenia reprezentacji (problem OOV, odnieś się do  k-gramów i n-gramów)?

Możemy zaobserwować następujące zalety:
* a

# Zadanie 2 - klasyfikacja (15 pkt)

Poniższy kod powinien wczytać i ztokenizować zbiór danych dot. analizy wydźwięku. Jeśli nie masz biblioteki `nltk` musisz ją zainstalować.

In [ ]:
from helpers import DataSet
training_set = DataSet(['tweets.txt'])

Poniżej znajdziesz przykład odczytu jednego tweeta z obiektu DataSet

In [ ]:
for i in training_set.tweets:
    print(i.text)
    print(i.tokens)
    print(i.clazz)
    break

Systemy IL często pracują z bardzo dużą liczbą cech, które są rzadkie np. cechy Bag-Of-Words, cechy n-gramowe itd. Powoduje to że klasyczna macierz przykłady uczące na cechy rośnie do bardzo dużych rozmiarów nawet dla małych zbiorów uczących (w sensie liczby przykładów). Ponadto samo przechowywanie w pamięci słownika mapującego konkretne słowa/n-gramy na indeksy kolumn macierzy może być bardzo kosztowne pamięciowo przy dużych rozmiarach słownika.

Istnieje jednak technika, która pozwala nam na ominięcie tej przeszkody: haszowanie cech. Opis tej techniki znajdziesz na stronie:  https://en.wikipedia.org/wiki/Feature_hashing Jest ona też implementowana w obiekcie `sklearn.feature_extraction.FeatureHasher`. Zapoznaj się z opisem techniki i wykonaj poniższe polecenia.

- Wykorzystując haszowanie cech wytrenuj wybrany klasyfikator na zbiorze uczącym dla cech Bag-of-words (możesz też spróbować cechy n-gramowe). Możesz wykorzystać gotową tokenizację we właściwości `.tokens`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

- Stwórz wykres zależności wybranej miary klasyfikacji od wymiarów macierzy danych (chodzi o liczbę cech do których haszujemy cechy oryginalne). Wystarczy przetestować kilka (>=4) wybranych wartości na skali logarytmicznej.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

 - Obserwując stworzony wykres - skomentuj. Jak dużo jakości klasyfikacji się traci (albo zyskuje?) korzystając z mniejszej liczby haszowanych cech? Często klasyfikatory bardzo dobrze działają nawet przy liczbie haszowanych cech dla których na pewno istnieją konflikty cech oryginalnych - jak myślisz dlaczego? (Pomyśl o interpretacji takich skonfliktowanych cech).

YOUR ANSWER HERE

 - W poprzednim zadaniu wczytałeś wynik grupowania Browna do pamięci. Wytrenuj klasyfikator na reprezentacji ,,Bag-of-clusters'' tj. w kolumnach zamiast słów/n-gramów będziesz miał grupy.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

- Podsumuj eksperymenty: poznałeś dwie możliwości ograniczenia liczby cech - zastąpienie słów ich grupami i haszowanie cech. Jakie są wady i zalety obydwu podejść?

YOUR ANSWER HERE